In [ ]:
! pip install requests 
! pip install pydub

In [ ]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

ElevenLabs_XI_API_KEY = os.environ.get("ElevenLabs_XI_API_KEY")

### Getting Voice IDs from the API

In [ ]:
import requests
import json

url = "https://api.elevenlabs.io/v1/voices"

headers = {
"Accept": "application/json",
"xi-api-key": ElevenLabs_XI_API_KEY,
"Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

data = response.json()

for voice in data['voices']:
    print(f"{voice['name']}; {voice['voice_id']}")


Grace; oWAxZDx7w5VEj9dCyTzz

### Getting Model List from the API

In [ ]:
import requests
import json

url = "https://api.elevenlabs.io/v1/models"

headers = {
"Accept": "application/json",
"xi-api-key": ElevenLabs_XI_API_KEY,
"Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

data = response.json()

print(data)

### Splitting audio file into multiple 4 minute junks.

In [ ]:
from pydub import AudioSegment
import os

if not os.path.exists('segments'):
    os.makedirs('segments')

audio = AudioSegment.from_mp3("input.mp3")

audio_length_ms = len(audio)

segment_length_ms = 240000

for i, start_time in enumerate(range(0, audio_length_ms, segment_length_ms)):
    end_time = start_time + segment_length_ms if start_time + segment_length_ms < audio_length_ms else audio_length_ms
    segment = audio[start_time:end_time]
    segment.export(f"segments/segment_{i}.mp3", format="mp3")


### Convert Speech

In [ ]:
import requests
import os

voice_id = "oWAxZDx7w5VEj9dCyTzz"
url = f"https://api.elevenlabs.io/v1/speech-to-speech/{voice_id}"
headers = {
    "xi-api-key": ElevenLabs_XI_API_KEY,
}
data = {
    "model_id": "eleven_multilingual_sts_v2",
    "voice_settings": "{\"similarity_boost\": 0.5, \"stability\": 0.5}",
}

# Iterate over each file in the segments folder
for i, filename in enumerate(os.listdir('segments')):
    if filename.endswith(".mp3"):
        with open(f"segments/{filename}", "rb") as audio_file:
            files = {
                "audio": audio_file,
            }
            response = requests.post(url, headers=headers, data=data, files=files)

            # Save the response content to a file
            with open(f'segment_output_{i}.mp3', 'wb') as f:
                f.write(response.content)


### Joining Segments and Saving the final audio file.

In [ ]:
from pydub import AudioSegment
import os

segments = []

# Iterate over each file in the converted-segments directory
for i, filename in sorted(enumerate(os.listdir('converted-segments')), key=lambda x: x[1]):
    if filename.startswith("segment_output_") and filename.endswith(".mp3"):
        # Load the audio file from the converted-segments directory and append it to the list
        segment = AudioSegment.from_mp3(f"converted-segments/{filename}")
        segments.append(segment)

# Concatenate all segments
final_audio = sum(segments)

# Export final audio
final_audio.export("final_output.mp3", format="mp3")
